In [44]:
import ast
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
#import plotly
#plotly.offline.init_notebook_mode(connected=True)
#import plotly.offline as py
import plotly.plotly as py
import plotly.tools as tls
import string
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from src.data import read_files
from collections import Counter
import itertools
from textblob import TextBlob
from langdetect import detect

import string
from googletrans import Translator
#from google.api_core.protobuf_helpers import get_messages
#from __future__ import absolute_import
from google.cloud import translate
from langdetect import detect
translator = Translator()
import math

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

import csv

In [45]:
#hotel = pd.DataFrame.from_csv('./data/hotel_train.csv')
hotel = pd.read_csv('./data/hotel.csv')
#foreign = pd.DataFrame.from_csv('./data/foreign.csv')
#hotel['words_clean'] = hotel['text_clean'].apply(lambda x: str(x).split(' '))
#hotel['words_clean'][foreign.index] = foreign['clean_words']
#hotel['text_clean'][foreign.index] = foreign['text_clean']
#hotel['phrases'][foreign.index] = foreign['phrases']

In [176]:
def count_fre(stars, ch):
    """
    count the frequency of 'ch': could be phrases or words
    corresponding to the 'stars': could be 1, 2, 3, 4, 5
    """
    p_list = list(itertools.chain(*list(hotel.loc[hotel['stars']==stars, ch])))
    p_count = pd.DataFrame(Counter(p_list).most_common())
    p = pd.DataFrame(p_count[p_count.columns[1]])
    p.index = p_count[p_count.columns[0]]
    p.columns = [str(stars)]
    return p

def str_to_list(s):
    """
    transform a string to a list, e.g. "[a,b,c]" to [a,b,c] 
    """
    s = s.replace("u'", "")
    s = s.replace("'", "")
    l = s[1:-1].split(',')
    return l

In [162]:
#counts of phrases
hotel['phrases'] = hotel['phrases'].apply(lambda x: str_to_list(x))
p = pd.concat([count_fre(1, 'phrases'), count_fre(2,'phrases'),count_fre(3,'phrases'),count_fre(4,'phrases'),count_fre(5,'phrases')],axis=1)
p = p.sort_values(by=['1'], ascending=False)
p = p.fillna(0)

In [177]:
#counts of words
w = pd.concat([count_fre(1, 'words_clean'), count_fre(2,'words_clean'),count_fre(3,'words_clean'),count_fre(4,'words_clean'),
               count_fre(5,'words_clean')],axis=1)
w = w.sort_values(by=['1'], ascending=False)
w = w.fillna(0)

In [145]:
def get_corr(row):
    x = [row['1'],row['2'],row['3'],row['4'],row['5']]
    return pearson_def(x,range(1,6))

In [ ]:
#######you could start from here if you wanna rerun the code########
w = pd.read_csv('./data/hotel_words_count.csv')
p = pd.read_csv('./data/hotel_phrase_count.csv')

In [21]:
from scipy.stats import spearmanr
def get_spearman_corr(row):
    """
    calculate the spearman's correlation coeffecient: for both linear and nonlinear relation
    """
    x = [row['1'],row['2'],row['3'],row['4'],row['5']]
    return spearmanr(x,range(1,6))[0]

In [49]:
p['correlation'] = p.apply(get_spearman_corr,axis=1)
w['correlation'] = w.apply(get_spearman_corr,axis=1)

In [54]:
p[(p['correlation']<-0.95) & (p['1']>90)].sort_values(by=['correlation'])

,Unnamed: 0,1,2,3,4,5,correlation
0,front desk,3222.0,1419.0,980.0,874.0,623.0,-1.000000
158,bad attitude,101.0,21.0,13.0,8.0,4.0,-1.000000
157,minutes line,101.0,46.0,28.0,18.0,13.0,-1.000000
146,toilet paper,104.0,45.0,35.0,34.0,15.0,-1.000000
143,switch rooms,105.0,50.0,25.0,22.0,9.0,-1.000000
115,poor customer service,119.0,31.0,9.0,6.0,0.0,-1.000000
114,wrong room,120.0,49.0,33.0,28.0,13.0,-1.000000
111,hotel manager,125.0,24.0,12.0,11.0,7.0,-1.000000
109,room didnt,125.0,74.0,72.0,58.0,26.0,-1.000000
96,didnt work,130.0,66.0,49.0,22.0,11.0,-1.000000


In [57]:
p[(p['correlation']>0.95) & (p['5']>100)].sort_values(by=['correlation'], ascending = False)

,Unnamed: 0,1,2,3,4,5,correlation
224,worth price,76.0,79.0,81.0,128.0,177.0,1.000000
266,friendly helpful,71.0,126.0,217.0,605.0,856.0,1.000000
591817,great tour,1.0,4.0,5.0,21.0,164.0,1.000000
128699,place awesome,1.0,4.0,5.0,31.0,128.0,1.000000
21301,amazing rooms,3.0,8.0,13.0,23.0,105.0,1.000000
18953,red rock canyon,3.0,6.0,13.0,33.0,110.0,1.000000
11499,amazing room,4.0,6.0,12.0,44.0,142.0,1.000000
10057,old montreal,5.0,10.0,29.0,92.0,109.0,1.000000
7824,great views,6.0,19.0,39.0,105.0,139.0,1.000000
7611,great prices,6.0,7.0,22.0,59.0,120.0,1.000000


In [60]:
w[(w['correlation']<-0.95) & (w['1']>2000)].sort_values(by=['correlation'])

,Unnamed: 0,1,2,3,4,5,correlation
9,told,30128.0,7451.0,3905.0,3181.0,3115.0,-1.0
541,ridiculous,2021.0,777.0,573.0,545.0,285.0,-1.0
537,joke,2059.0,490.0,272.0,181.0,170.0,-1.0
524,carpet,2086.0,1160.0,859.0,585.0,356.0,-1.0
493,Horrible,2225.0,144.0,58.0,20.0,9.0,-1.0
487,stated,2252.0,514.0,351.0,281.0,214.0,-1.0
486,waste,2258.0,552.0,324.0,294.0,272.0,-1.0
483,somewhere,2265.0,1118.0,1007.0,913.0,609.0,-1.0
472,fix,2309.0,803.0,689.0,607.0,435.0,-1.0
462,smelled,2344.0,1154.0,770.0,654.0,447.0,-1.0


In [62]:
w[(w['correlation']>0.95) & (w['5']>1500)].sort_values(by=['correlation'], ascending = False)

,Unnamed: 0,1,2,3,4,5,correlation
156,great,5940.0,6404.0,11878.0,31735.0,42094.0,1.0
394,friendly,2717.0,3097.0,5439.0,11849.0,16019.0,1.0
5411,knowledgeable,89.0,91.0,132.0,443.0,1977.0,1.0
5364,notch,90.0,116.0,212.0,713.0,1775.0,1.0
5232,outstanding,94.0,108.0,224.0,592.0,1584.0,1.0
4947,Beautiful,102.0,151.0,318.0,932.0,1504.0,1.0
4239,attentive,132.0,248.0,429.0,1345.0,1953.0,1.0
4108,gorgeous,139.0,211.0,398.0,1369.0,2593.0,1.0
4014,delicious,144.0,305.0,739.0,2448.0,4205.0,1.0
3342,chocolate,194.0,254.0,473.0,1132.0,1530.0,1.0
